### Tensorflow 1.x Object Detection Setup



### GPU check for Google Colab

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
!nvidia-smi

### Checking the libraries comes preinstalled with Google Colab

In [ ]:
!pip freeze

### Uninstalling the latest version of Tensorflow which is 2.4.1

In [ ]:
!pip uninstall tensorflow==2.7.0

### Installing the version of tensorflow-gpu 1.14.0 which we need for TF 1.x Object Detectection

In [ ]:
!pip install tensorflow-gpu==1.14.0

### Importing tensorflow

In [ ]:
import tensorflow as tf

### Tensorflow version check 

In [ ]:
print(tf.__version__)

### GPU check

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

### Build with CUDA Test

In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

### Tensorflow 1.x Object Detection Setup

In [ ]:
# Navigate to Google Drive
%cd /content/drive/MyDrive/TFOD1.x setup
!pwd

In [ ]:
# Download Repo https://github.com/tensorflow/models/tree/v1.13.0
!wget https://github.com/tensorflow/models/archive/v1.13.0.zip

In [ ]:
#Unzipping the zip file
!unzip  v1.13.0.zip

In [ ]:
!ls
# Rename models-1.13.0 to models
!mv models-1.13.0/ models


In [ ]:
!pwd

In [ ]:
%cd models/research/
!pwd

In [ ]:
%cd object_detection/
!pwd

In [ ]:
%cd /content/drive/MyDrive/TFOD1.x setup/models/research
!ls

In [ ]:
!python setup.py  install 

In [ ]:
### Protos Conversion to Python
%cd /content/drive/MyDrive/TFOD1.x setup/models/research
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
  !pwd


In [ ]:
tf.__version__
!pwd

### Imports



In [ ]:
%cd /content/drive/MyDrive/TFOD1.x setup/models/research

In [ ]:
!pwd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### XML to CSV


In [ ]:
!python xml_to_csv.py

#### From CSV to TF record

In [ ]:
!python generate_tfrecord.py --csv_input=images/train_labels.csv  --image_dir=images/train --output_path=train.record


In [ ]:
!python generate_tfrecord.py --csv_input=images/test_labels.csv  --image_dir=images/test --output_path=test.record


In [ ]:
!pwd
tf.__version__

### Training procedure

In [ ]:
!python train.py --logtostderr --train_dir=/content/drive/MyDrive/TFOD1.x setup/models/research/training/ --pipeline_config_path=object_detection/samples/configs/faster_rcnn_inception_v2_coco.config

### Export Frozen Inference Graph
from ckpt (checkpoint) file (trained model on custom data) load Inference Graph.

In [ ]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path=object_detection/samples/configs/faster_rcnn_inception_v2_coco.config --trained_checkpoint_prefix=/content/drive/MyDrive/TFOD1.x/model.ckpt-5000 --output_directory=my_exported_model 

### Inferencing with our trained model.



In [ ]:
!pwd

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib  ##download files
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion ## 
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

In [ ]:
# This is needed to display the images.
%matplotlib inline

In [ ]:
%cd /content/drive/My Drive/TFOD1.x setup/models/research/object_detection


### Object detection imports
Here are the imports from the object detection module.



In [ ]:
from object_detection.utils import label_map_util   ##dict of classes 

from object_detection.utils import visualization_utils as vis_util    ## draw sq boxes, confidence 

### Model preparation
Variables
Any model exported using the export_inference_graph.py tool can be loaded here simply by changing PATH_TO_FROZEN_GRAPH to point to a new .pb file.

By default we use an "SSD with Mobilenet" model here. See the detection model zoo for a list of other models that can be run out-of-the-box with varying speeds and accuracies.




In [ ]:
# What model to download.
MODEL_NAME = 'my_exported_model'
# MODEL_FILE = MODEL_NAME + '.tar.gz'
# DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training', 'labelmap.pbtxt')

### Download Model



In [ ]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())


### Load a (frozen) Tensorflow model into memory.


In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

### Loading label map
Label maps map indices to category names, so that when our convolution network predicts 5, we know that this corresponds to airplane. Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine




In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


### Helper code


In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

### Detection

In [ ]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(5, 8) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [ ]:
%matplotlib inline
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)
